In [1]:
from google.cloud import bigquery
from google.cloud import bigquery_storage
from google.oauth2 import service_account
from google.cloud import storage
import pandas as pd
import requests

SERVICE_ACCOUNT_FILE = 'key.json'
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)
bqclient = bigquery.Client(project='iotube-analytics', credentials=credentials)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)

In [2]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    '''
    Uploads data to Google Cloud Storage bucket
    '''
    storage_client = storage.Client.from_service_account_json(SERVICE_ACCOUNT_FILE)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    
    blob.upload_from_string(source_file_name, 'text/csv')
    blob.make_public()

    print("File uploaded to {}.".format(destination_blob_name))

In [14]:
def txnsStatsByDate():
    '''
    Fetches network level stats of networks - IoTeX, Polygon, Ethereum and Zilliqa
    '''
    
    query_string = """SELECT COUNT(DISTINCT sender) AS Addresses, 
    COUNT(*) AS Transactions, 
    DATE(timestamp) as Date,
    "IoTeX" as Network
    FROM `public-data-finance.crypto_iotex.actions` 
    WHERE DATE(timestamp) >= '2021-03-01' 
    GROUP BY Date
    ORDER BY Date DESC
    """

    txnByDateIotex = (
        bqclient.query(query_string)
        .result()
        .to_dataframe(bqstorage_client=bqstorageclient)
    )
    
    query_string = """SELECT COUNT(DISTINCT from_address) AS Addresses, 
    COUNT(*) AS Transactions, 
    DATE(block_timestamp) as Date,
    "Polygon" as Network
    FROM `public-data-finance.crypto_polygon.transactions`
    WHERE block_timestamp >= '2021-03-01' 
    GROUP BY Date
    ORDER BY Date DESC
    """

    txnByDatePolygon = (
        bqclient.query(query_string)
        .result()
        .to_dataframe(bqstorage_client=bqstorageclient)
    )
    txnByDate = txnByDateIotex.append(txnByDatePolygon)

    query_string = """SELECT COUNT(DISTINCT sender) AS Addresses, 
    COUNT(*) AS Transactions, 
    DATE(block_timestamp) as Date,
    "Zilliqa" as Network
    FROM `public-data-finance.crypto_zilliqa.transactions`
    WHERE block_timestamp >= '2021-03-01' 
    GROUP BY Date
    ORDER BY Date DESC 
    """
    txnByDateZilliqa = (
        bqclient.query(query_string)
        .result()
        .to_dataframe(bqstorage_client=bqstorageclient)
    )
    txnByDate = txnByDate.append(txnByDateZilliqa)
    
    query_string = """SELECT COUNT(DISTINCT from_address) AS Addresses, 
    COUNT(*) AS Transactions, 
    DATE(block_timestamp) as Date,
    "Ethereum" as Network
    FROM `bigquery-public-data.crypto_ethereum.transactions`
    WHERE block_timestamp >= '2021-03-01' 
    GROUP BY Date
    ORDER BY Date DESC
    """
    txnByDateEth = (
        bqclient.query(query_string)
        .result()
        .to_dataframe(bqstorage_client=bqstorageclient)
    )
    txnByDate = txnByDate.append(txnByDateEth)

    upload_blob("iotube", txnByDate.to_csv(), "txnStatsByDate")

In [15]:
def getHourData():
    '''
    Returns actions count by hour - yearly and last 24 hours
    '''
    
    query_string = """SELECT COUNT(*) as NoOfActions, 
    EXTRACT(HOUR FROM timestamp) as Hour
    FROM `public-data-finance.crypto_iotex.actions` 
    GROUP BY Hour
    ORDER BY Hour
    """
    yearHourData = (
        bqclient.query(query_string)
        .result()
        .to_dataframe(bqstorage_client=bqstorageclient)
    )

    upload_blob("iotube", yearHourData.to_csv(), "yearHourData")

    query_string = """
    SELECT COUNT(*) as NoOfActions, 
    EXTRACT(HOUR FROM timestamp) as Hour
    FROM `public-data-finance.crypto_iotex.actions` 
    WHERE DATE(timestamp) >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)
    GROUP BY Hour
    ORDER BY Hour
    """
    newHourData = (
        bqclient.query(query_string)
        .result()
        .to_dataframe(bqstorage_client=bqstorageclient)
    )

    upload_blob("iotube", newHourData.to_csv(), "newHourData")

In [19]:
def getNetworkStats():
    '''
    Get network statistics for the networks on a daily basis
    '''
    
    # IoTeX
    query_string = """SELECT DATE(timestamp) as Date, 
    COUNT(`hash`) AS NoOfTxn, 
    SUM(gas_consumed) as TotalGasUsed,
    AVG(gas_price * POW(10, -18)) AS AvgGasPrice,
    SUM(gas_consumed) * AVG(gas_price * POW(10, -18)) AS TotalTxnFee,
    SUM(gas_consumed) * AVG(gas_price * POW(10, -18)) / COUNT(`hash`) AS AvgTxnFee,
    "IoTeX" AS Network
    FROM `public-data-finance.crypto_iotex.actions` 
    WHERE DATE(timestamp) > "2021-01-01" 
    GROUP BY Date
    ORDER BY Date ASC
    """

    txnByDateIotex = (
        bqclient.query(query_string)
        .result()
        .to_dataframe(bqstorage_client=bqstorageclient)
    )
    iotx = pd.read_csv('https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=IOTX&market=CNY&apikey=3NXQI7IVLOCKQTUO&datatype=csv')
    txnByDateIotex = pd.concat([txnByDateIotex, iotx], axis=1, join="inner")
    txnByDateIotex.sort_values(by=['Date'], inplace=True)
    txnByDateIotex['TotalTxnFeeUSD'] = txnByDateIotex['TotalTxnFee'] * txnByDateIotex['close (USD)']
    txnByDateIotex['CummulativeTotalTxnFee'] = txnByDateIotex['TotalTxnFeeUSD'].cumsum()
    txnByDateIotex['CummulativeTotalGasUsed'] = txnByDateIotex['TotalGasUsed'].cumsum()

    # Ethereum
    query_string = """SELECT DATE(block_timestamp) as Date, 
    COUNT(`hash`) AS NoOfTxn, 
    SUM(receipt_gas_used) as TotalGasUsed,
    AVG(gas_price * POW(10, -18)) AS AvgGasPrice,
    SUM(receipt_gas_used) * AVG(gas_price * POW(10, -18)) AS TotalTxnFee,
    SUM(receipt_gas_used) * AVG(gas_price * POW(10, -18)) / COUNT(`hash`) AS AvgTxnFee,
    "Ethereum" AS Network
    FROM bigquery-public-data.crypto_ethereum.transactions 
    WHERE DATE(block_timestamp) > "2021-01-01" 
    GROUP BY Date
    ORDER BY Date ASC
    """
    txnByDateEth = (
        bqclient.query(query_string)
        .result()
        .to_dataframe(bqstorage_client=bqstorageclient)
    )
    eth = pd.read_csv('https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=ETH&market=CNY&apikey=3NXQI7IVLOCKQTUO&datatype=csv')
    txnByDateEth = pd.concat([txnByDateEth, eth], axis=1, join="inner")
    txnByDateEth.sort_values(by=['Date'], inplace=True)
    txnByDateEth['TotalTxnFeeUSD'] = txnByDateEth['TotalTxnFee'] * txnByDateEth['close (USD)']
    txnByDateEth['CummulativeTotalTxnFee'] = txnByDateEth['TotalTxnFeeUSD'].cumsum()
    txnByDateEth['CummulativeTotalGasUsed'] = txnByDateEth['TotalGasUsed'].cumsum()

    txnByDate = txnByDateIotex.append(txnByDateEth)
    txnByDate['AvgTxnFeeUSD'] = txnByDate['AvgTxnFee'] * txnByDate['close (USD)']

    upload_blob("iotube", txnByDate.to_csv(), "networkStatsNew")

    return txnByDate

In [26]:
df = pd.read_csv('https://storage.googleapis.com/iotube/txnStatsByDate')
txnsByDate = pd.read_json(df.to_json(date_format='iso', orient='split'), orient='split')
txnsByDate = txnsByDate[txnsByDate.Date < pd.Timestamp('today').floor('D')]
txnsByDate

,Unnamed: 0,Addresses,Transactions,Date,Network
1,1,1266,44229,2021-07-05,IoTeX
2,2,1191,44099,2021-07-04,IoTeX
3,3,1350,44046,2021-07-03,IoTeX
4,4,1313,44184,2021-07-02,IoTeX
5,5,1215,44243,2021-07-01,IoTeX
...,...,...,...,...,...
506,123,331034,1208640,2021-03-05,Ethereum
507,124,367971,1234903,2021-03-04,Ethereum
508,125,356110,1235048,2021-03-03,Ethereum
509,126,385980,1240182,2021-03-02,Ethereum
